<a href="https://colab.research.google.com/github/MZiaAfzal71/Average_Weighted_Path_Vector/blob/main/Data%20Files/Models/XGBoostModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/MZiaAfzal71/Average_Weighted_Path_Vector.git
%cd Average_Weighted_Path_Vector/Data\ Files

Cloning into 'Average_Weighted_Path_Vector'...
remote: Enumerating objects: 668, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 668 (delta 63), reused 8 (delta 8), pack-reused 585 (from 1)
Receiving objects: 100% (668/668), 30.59 MiB | 20.13 MiB/s, done.
Resolving deltas: 100% (217/217), done.
/content/Average_Weighted_Path_Vector/Data Files


In [46]:
!pip install osfclient
import os
from osfclient.api import OSF
from subprocess import run

# Replace with your OSF project ID
project_id = "p5ga2"   # e.g. from https://osf.io/abcd3/
osf = OSF()
project = osf.project(project_id)
store = project.storage("osfstorage")

desc_folder = []
for fold in store.folders:
    if fold.path.strip("/") == "Descriptors Data":
        desc_folder = fold
        break

# Download all files and keep folder structure
for f in desc_folder.files:
    local_path = f.path.strip("/")            # keep folders
    local_dir = os.path.dirname(local_path)   # extract dir
    if local_dir and not os.path.exists(local_dir):
        os.makedirs(local_dir, exist_ok=True) # create dirs if missing
    with open(local_path, "wb") as out:
        f.write_to(out)
    if local_path.endswith(".zip"):
      command = f"unzip '{local_path}' -d '{local_dir}'"
      run(command, shell=True)
      print(f"\nUnzipped {local_path} -> {local_dir}")
      continue
    print(f"Downloaded {f.path} -> {local_path}")

100%|██████████| 23.8M/23.8M [00:02<00:00, 9.69Mbytes/s]



Unzipped Descriptors Data/Descriptors Data.zip -> Descriptors Data


In [3]:
from xgboost import XGBRegressor # The model to learn Fusion points of Alkanes
from sklearn.metrics import mean_absolute_error, r2_score, root_mean_squared_error  # To find errors
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd

In [4]:
# === Paths & Settings ===
input_dir = "Descriptors Data"
output_dir = "XGBoost Results"
os.makedirs(output_dir, exist_ok=True)

property_sheets = ["Log VP", "MP", "BP", "LogBCF", "LogS", "LogP"]
desc_keys = ["MACCS", "Morgan", "pwav"]

# === Main Loop ===
for prop in property_sheets:
    for dsk in desc_keys:
        # Input file
        input_file = os.path.join(input_dir, f"{prop}_{dsk}.parquet")
        print(f"\n🔹 Processing: {input_file}")

        # Read data
        df = pd.read_parquet(input_file)
        prop_pred = f"{prop}-Measured"

        # Feature matrix & target
        X = df.iloc[:, 9:]
        y = df[prop_pred]

        # Train/test split
        train_idx = df[df["Training/Test"] == "Training"].index.tolist()
        test_idx  = df[df["Training/Test"] == "Test"].index.tolist()

        X_train, X_valid = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_valid = y.iloc[train_idx], y.iloc[test_idx]

        # Define model
        model = XGBRegressor(random_state=42)
        # Alternative (your tuned version, uncomment if needed)
        # model = XGBRegressor(
        #     n_estimators=100, eta=0.081, max_depth=3, gamma=1, min_child_weight=1,
        #     colsample_bytree=0.95, reg_lambda=1, reg_alpha=0, max_delta_step=0,
        #     subsample=1, tree_method="hist", scale_pos_weight=1,
        #     grow_policy="lossguide", max_leaves=0, max_bin=80,
        #     num_parallel_tree=1, random_state=42
        # )

        # Train model
        model.fit(X_train, y_train)

        # Predict on validation set
        y_pred = model.predict(X_valid)

        # === Stats & Metrics ===
        errors = abs(y_valid - y_pred)
        stats_summary = errors.describe()

        # Print metrics
        print("📊 Error Summary:")
        print(stats_summary)
        print(f"   ➤ R²   : {r2_score(y_valid, y_pred):.4f}")
        print(f"   ➤ MAE  : {mean_absolute_error(y_valid, y_pred):.4f}")
        print(f"   ➤ RMSE : {root_mean_squared_error(y_valid, y_pred):.4f}")

        # === Full Predictions ===
        y_pred_all = model.predict(X)
        df_results = df.iloc[:, :9].copy()
        df_results[f"{prop} {dsk} Predictions"] = y_pred_all

        # Save results
        output_file = os.path.join(output_dir, f"{prop}_{dsk}_xgboost.xlsx")
        df_results.to_excel(output_file, index=False)

        print(f"✅ Results saved: {output_file}")

print("\n🎉 All files have been processed successfully!")



🔹 Processing: Descriptors Data/Log VP_MACCS.parquet
📊 Error Summary:
count    679.000000
mean       1.039008
std        1.078119
min        0.004539
25%        0.292238
50%        0.689126
75%        1.399440
max        6.914137
Name: Log VP-Measured, dtype: float64
   ➤ R²   : 0.8228
   ➤ MAE  : 1.0390
   ➤ RMSE : 1.4967
✅ Results saved: XGBoost Results/Log VP_MACCS_xgboost.xlsx

🔹 Processing: Descriptors Data/Log VP_Morgan.parquet
📊 Error Summary:
count    679.000000
mean       1.346268
std        1.239952
min        0.002498
25%        0.445449
50%        0.969699
75%        1.970971
max        7.966365
Name: Log VP-Measured, dtype: float64
   ➤ R²   : 0.7352
   ➤ MAE  : 1.3463
   ➤ RMSE : 1.8297
✅ Results saved: XGBoost Results/Log VP_Morgan_xgboost.xlsx

🔹 Processing: Descriptors Data/Log VP_pwav.parquet
📊 Error Summary:
count    679.000000
mean       0.703033
std        0.802072
min        0.001165
25%        0.142756
50%        0.426437
75%        0.934681
max        5.525071
N